# Calculate Alpha, Beta and CAPM

In [39]:
# import libraries

import os.path
import numpy as np
import pandas as pd
import pandas_datareader as web
import hvplot.pandas
from pathlib import Path
from datetime import date

%matplotlib inline


In [2]:
# intitialize variables

index_stocks = ['AGG', 'SPY', "QQQ"]
picked_stocks = ['ACB', 'T', 'GOOG', 'BA']
random_stocks = ['GAIA', 'FMS', 'RC']

combined_stocks = index_stocks + picked_stocks + random_stocks
combined_stocks.sort()

start = pd.to_datetime('2015-10-01')
end = pd.to_datetime('2019-10-01')


In [3]:
# Create function to pull finance data, and save to csv, then 
stock_df = f"./csv/{combined_stocks[0]}-to-{combined_stocks[-1]}-from-{start.date()}-to-{end.date()}.csv"

if os.path.exists(stock_df): #if the csv file exists in directory, load file

    print(f"{stock_df} exists in this directory")

    stocks = pd.read_csv(stock_df, parse_dates=True, index_col="Date")

else: #otherwise pull fresh data

    print(f"Expected file does not exist, so a new file was created at {stock_df}.")

    stock_data = web.DataReader(combined_stocks, 'yahoo', start, end)

    stocks = stock_data["Close"]

    stocks.to_csv(f"./csv/{combined_stocks[0]}-to-{combined_stocks[-1]}-from-{start.date()}-to-{end.date()}.csv")


./csv/ACB-to-T-from-2015-10-01-to-2019-10-01.csv exists in this directory


In [54]:
# Pull and average historical return rate for date range matching stocks

bond_rate = web.DataReader('^TNX', 'yahoo', start, end)
gov_bond_rate = bond_rate['Close'].mean()

In [4]:
# calculate daily change and drop the resulting first null row
stocks_daily_change = stocks.pct_change().dropna()

In [5]:
# create several smaller dataframes to contain the different groups of stocks picked
index_daily_change = stocks_daily_change[index_stocks]
picked_daily_change = stocks_daily_change[picked_stocks]
random_daily_change = stocks_daily_change[random_stocks]


In [6]:
# Create a variance, covariance and beta function

def var_cov_beta(stock_df, index_df):
    """Calculate the beta of a given stock against a given index"""

    variance = stock_df.var()
    covariance = stock_df.cov(index_df)
    beta = covariance / variance

    return beta
    # print(f"{stock_df.name} has a beta of {beta} when measured against the {index_df.name} index.")

# Create a rolling beta dataframe
def var_cov_beta_rolling(stock_df, index_df, window):
    """Requires a window parameter (integer in days) and Calculate a rolling beta for a given stock and a given index."""

    rolling_variance = stock_df.rolling(window=window).var()
    rolling_covariance = stock_df.rolling(window=window).cov(index_df)

    rolling_beta = rolling_covariance / rolling_variance

    # Plot rolling beta in hvplot
    return rolling_beta.hvplot(figsize=(20, 10), title=f'Rolling {window}-Day Beta of {stock_df.name} and index {index_df.name}')


In [7]:
var_cov_beta(stocks_daily_change['GOOG'], stocks_daily_change['SPY'])

0.4053614882481492

In [8]:
var_cov_beta_rolling(stocks_daily_change['GOOG'], stocks_daily_change['SPY'], 50)

:Curve   [Date]   (0)

In [9]:
# create covariance for each picked stock and one chosen index fund with a nested forLoop

# for x in range(0, len(combined_stocks)):

#     cov_var = combined_stocks[x]

#     variance = stocks_daily_change[cov_var].var()

#     print(f"\nThe variance of {cov_var} is: {variance}")

#     cov_var2 = index_stocks[0]

#     covariance = stocks_daily_change[cov_var].cov(index_daily_change[cov_var2])

#     print(f"The covariance between {cov_var} and {cov_var2} is: {covariance}")

#     print(f"The Beta of {cov_var} and {cov_var2} is {covariance / variance}")
    

In [10]:
# Use Numpy's built-in polynomial-fit feature to find the alpha and beta of a stock and an index

index_s = stocks_daily_change['SPY']
stock_s = stocks_daily_change['GOOG']

In [11]:
# Using Numpy's older 'polyfit' method to calculate beta alpha

beta, alpha = np.polyfit(stock_s, index_s, 1)
print('Beta for {} stock is = {} and alpha is = {}'.format(stock_s.name, beta, alpha))

# # Using Numpy's newer polynomial.fit method
# beta, alpha = np.polynomial.polynomial.Polynomial.fit(index_s, index_s, 1)
# print('Beta for {} stock is = {} and alpha is = {}'.format(stock_s.name, beta, alpha))

# Results varied between methods, uncertain why, using older method because beta matches manual calculations
# Further research required for np.polyfit and polynomial.fit methods and their varied results


Beta for GOOG stock is = 0.4053614882481487 and alpha is = 0.00013953344476051122


In [58]:
# calculate risk free rate
# enter following variables as percentages
def risk_free_rate():
    ten_yr_gov_bond = gov_bond_rate
    inflation_rate = .009
    market_return = .06
    beta = var_cov_beta(picked_daily_change['T'], index_daily_change['AGG'])

    risk_free_rate = round((((1 + ten_yr_gov_bond)/(1 + inflation_rate))-1), 3)

    return risk_free_rate
    # print(f"Your risk free rate is: {risk_free_rate}%")

risk_free_rate()

2.296

In [59]:
# Define CAPM formula
annual_market_return = (stocks_daily_change['SPY'].mean() * len(stocks_daily_change) *100)

def capm(risk_free_rate, beta, market_risk_premium):
    expected_return = risk_free_rate + (beta * market_risk_premium)
    return expected_return



In [60]:
capm(risk_free_rate(), beta, annual_market_return)

20.869918065713886

In [61]:
stocks['GOOG'][-1] - stocks['GOOG'][0]

593.8099975585938